# Dice Game

**Aim of this notebook:** apply the Dynamic Programming Equation (DPE) to find the optimal strategy of a game.

Let's play a very simple game. Here are the rules:
* It's a 3-turn game.
* The player starts with 0 points.
* In each turn, the player has a choice: either increase score by one unit, or roll a 6-sided die. If the die lands on 2, 4, or 6, the total score resets to 0. If it lands on 1, 3, or 5, the score from the die is added to the previous total.
* The goal is to score as many points as possible in 3 turns. What is the best strategy to win?

## Solution

Let $X_0 = 0$ represent the initial number of points, $X_1$ the points after the first turn, $X_2$ the points after the second turn, and $X_3$ the points after the third turn, which is the final score that we aim to maximize.

In each turn, the player has a decision to make. On turn $n \ (1 \leq n \leq 3)$, the player can either choose to move forward (denoted $U_n = 0$) or roll the die (denoted $U_n = 1$). The result of the die roll in each turn is represented as $W_n$.

We have the following formula for each turn $n$:

$$
X_{n+1} = \chi(U_n = 0) (X_n + 1) + \chi(U_n = 1) \chi(W_n \in \{1, 3, 5\}) (X_n + W_n)
$$

This can be rewritten as a function of the current state $X_n$, decision $U_n$, and die result $W_n$:

$$
X_{n+1} = f_n(X_n, U_n, W_n)
$$

We are looking for a strategy function $\sigma$ such that:

$$
U_n = \sigma(X_0, W_0, \ldots, X_{n-1}, W_{n-1}, X_n)
$$

that maximizes the expected final score:

$$
J(X, U) = \mathbb{E}(X_3)
$$


## State Transitions

Let's calculate the possible state transitions for each turn. For the decision $U_n = 0$ (move forward), we have:

$$
P(f_n(x, 0, W_n) = x+1) = 1
$$

For the decision $U_n = 1$ (roll the die), the probabilities of the next state are as follows:

$$
P(f_n(x, 1, W_n) = 0) = \frac{1}{2}
$$
$$
P(f_n(x, 1, W_n) = x+1) = \frac{1}{6}
$$
$$
P(f_n(x, 1, W_n) = x+3) = \frac{1}{6}
$$
$$
P(f_n(x, 1, W_n) = x+5) = \frac{1}{6}
$$

Note that the process is stationary: we could represent it by a Markov graph.

## Dynamic Programming

We start with the value function at the end of the game:

$$
v_3(x) = x
$$

Now, let's calculate the value function for turn 2 using the **Dynamic Programming Equation (DPE)** :

$$
v_2(x) = \max\left( P(f_2(x, 0, W_2) = x+1) v_3(x+1),
P(f_2(x, 1, W_2) = 0) v_3(0) + P(f_2(x, 1, W_2) = x+1) v_3(x+1) + P(f_2(x, 1, W_2) = x+3) v_3(x+3) + P(f_2(x, 1, W_2) = x+5) v_3(x+5)\right)
$$

Simplifying:

$$
v_2(x) = \max(x+1, \frac{1}{6}(x+1) + \frac{1}{6}(x+3) + \frac{1}{6}(x+5))
$$

This becomes:

$$
v_2(x) = \max(x+1, \frac{1}{2}x + \frac{3}{2})
$$

Thus, we have:

$$
v_2(x) = \begin{cases}
\frac{1}{2}x + \frac{3}{2} & \text{if } x = 0 \\
x + 1 & \text{if } x \geq 1
\end{cases}
$$

Next, for turn 1, we apply the dynamic programming equation (DPE) again:

$$
v_1(x) = \max\left( P(f_1(x, 0, W_1) = x+1) v_2(x+1),
P(f_1(x, 1, W_1) = 0) v_2(0) + P(f_1(x, 1, W_1) = x+1) v_2(x+1) + P(f_1(x, 1, W_1) = x+3) v_2(x+3) + P(f_1(x, 1, W_1) = x+5) v_2(x+5) \right)
$$

Thus:

$$
v_1(x) = \max(x+2, \frac{1}{2} \cdot \frac{3}{2} + \frac{1}{6}(x+2) + \frac{1}{6}(x+4) + \frac{1}{6}(x+6))
$$

Simplifying:

$$
v_1(x) = \max(x+2, \frac{1}{2}x + \frac{11}{4})
$$

Thus:

$$
v_1(x) = \begin{cases}
\frac{1}{2}x + \frac{11}{4} & \text{if } x \leq 1 \\
x + 2 & \text{if } x \geq 2
\end{cases}
$$

Finally, we apply the DPE one last time for turn 0:

$$
v_0(0) = \max\left( P(f_0(0, 0, W_0) = 1) v_1(1),
P(f_0(0, 1, W_0) = 0) v_2(0) + P(f_0(0, 1, W_0) = 1) v_2(1) + P(f_0(0, 1, W_0) = 3) v_2(3) + P(f_0(0, 1, W_0) = 5) v_2(5)\right)
$$

Simplifying:

$$
v_0(0) = \max\left(\frac{13}{4}, \frac{1}{2} \cdot \frac{11}{4} + \frac{1}{6} \cdot \frac{13}{4} + \frac{1}{6} \cdot 5 + \frac{1}{6} \cdot 7\right) = \frac{47}{12} \approx 3.92
$$




## Optimal Strategy

So, according to the above calculations, the best strategy for the player is as follows:
* On the first turn: Roll the die.
* On the second turn: If you have 0 or 1 point, roll the die. Otherwise, move forward 1 space without rolling.
* On the third turn: If you have 0 points, roll the die. Otherwise, move forward 1 space without rolling.

Remark: this is a "feedback policy" because it depends only on the last state

## Simulation

We will now simulate this game $10^{6}$ times and observe the average score obtained by following the best strategy (we expect to find an average of 3.92).

In [11]:
import random
import numpy as np

def game_best_strategy():
  X0,X1,X2,X3=0,0,0,0
  d0=random.randint(1,6)
  if d0%2==0:
    X1=0
  else:
    X1=d0
  if X1==0 or X1==1:
    d1=random.randint(1,6)
    if d1%2==0:
      X2=0
    else:
      X2=X1+d1
  else:
    X2=X1+1
  if X2==0:
    d2=random.randint(1,6)
    if d2%2==0:
      X3=0
    else:
      X3=X2+d2
  else:
    X3=X2+1
  return X3

scores=np.zeros(10**6)

for k in range(10**6):
  scores[k]=game_best_strategy()

print(scores.mean())

3.915758


Let's try a slightly different strategy: we expect a lower result.

In [12]:
import random
import numpy as np

def game_slightly_different_strategy():
  X0,X1,X2,X3=0,0,0,0
  d0=random.randint(1,6)
  if d0%2==0:
    X1=0
  else:
    X1=d0
  if X1==0 or X1==1 :
    d1=random.randint(1,6)
    if d1%2==0:
      X2=0
    else:
      X2=X1+d1
  else:
    X2=X1+1
  if X2==0 and X2==1 : # we added X2==1 here
    d2=random.randint(1,6)
    if d2%2==0:
      X3=0
    else:
      X3=X2+d2
  else:
    X3=X2+1
  return X3

scores=np.zeros(10**6)

for k in range(10**6):
  scores[k]=game_slightly_different_strategy()

print(scores.mean())

3.75141


Finally, let's try the strategy where the die is rolled each time.

In [13]:
import random
import numpy as np

def game_pure_random_strategy():
  X0,X1,X2,X3=0,0,0,0
  d0=random.randint(1,6)
  if d0%2==0:
    X1=0
  else:
    X1=d0
  d1=random.randint(1,6)
  if d1%2==0:
    X2=0
  else:
    X2=X1+d1
  d2=random.randint(1,6)
  if d2%2==0:
    X3=0
  else:
    X3=X2+d2
  return X3

scores=np.zeros(10**6)

for k in range(10**6):
  scores[k]=game_pure_random_strategy()

print(scores.mean())

2.633702


The strategy of using the die every time is not the best one.